In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
sns.set(font_scale=1.5)

**************************************************

Testing param sweep -- parcel wp

**************************************************

In [2]:
df = pd.read_pickle("testing_sweep.pkl",compression='bz2')
df['NRecovered'] = df['FracRecovered']*df['NStaff']
df['NInfected'] = df['NRecovered']-1
df['outbreak'] = (df['NRecovered'] > 4)
df.columns

Index(['Group', 'NStaff', 'Iteration', 'FracRecovered', 'FomiteInfectionFrac',
       'CohortInfectionFrac', 'RandContactInfectionFrac', 'PairInfectionFrac',
       'RoomInfectionFrac', 'ExtIntroFrac', 'CustIntroFrac', 'CarShareInfFrac',
       'HouseShareInfFrac', 'CustomersInfected', 'IsolatorsFrac',
       'SympIsolatorsFrac', 'FPIsolatorsFrac', 'TPSympIsolatorsFrac',
       'TPAsympIsolatorsFrac', 'IndexCaseInfections', 'IndexCaseViralLoad',
       'IndexCaseInfectivity', 'OverallOutbreakLength', 'is_testing',
       'protocol', 'tperiod', 'specificity', 'new_comply_prob', 'tD',
       'SimType', 'TeamTimes', 'phi', 'TeamDistances', 'InfInit',
       'HouseShareFactor', 'CarShareFactor', 'Psusc', 'delay', 'Pisol',
       'NDteams', 'is_cohorts', 'p_contact', 'test_pause', 'NOteams',
       'TeamsOutside', 'NLteams', 'NRecovered', 'NInfected', 'outbreak'],
      dtype='object')

In [3]:
dfbaseline = df.loc[lambda DF: (DF['protocol'] == 'No testing')]
dfball = dfbaseline.loc[lambda DF: (DF['Group'] == 'All')]

In [4]:
dftest = df.loc[lambda DF: (DF['protocol'] != 'No testing')]
TProtos = np.unique(dftest["protocol"])
Delays = np.unique(dftest["delay"])
NCP = np.unique(dftest["new_comply_prob"])
Tperiod = np.unique(dftest["tperiod"])

print('II = ', np.unique(dftest["InfInit"]))
print('TD = ', np.unique(dftest["tD"]))
print('Phi = ', np.unique(dftest['phi']))
print('Pisol = ', np.unique(dftest['Pisol']))
print('Specificity = ', np.unique(dftest['specificity']))
print('Random contact rate = ', np.unique(dftest['p_contact']))
print('Friend contact prob = ', np.unique(dftest['p_friend_contact']))
print('Staff totals = ', np.unique(dftest['NStaff']))

TypeError: '<' not supported between instances of 'str' and 'int'

need likewise no testing case

In [ ]:
dfall = dftest.loc[dftest['Group']=='All',:]
dfall['scenario'] = np.zeros(len(dfall),dtype=np.object)
dfall.loc[lambda df: (df["protocol"]=='LFD_mass_testing'),'scenario'] = "LFD"
dfall.loc[lambda df: (df["protocol"]=='PCR_mass_testing') &\
                     (df["delay"]==0),'scenario'] = "PCR - 0 delay"
dfall.loc[lambda df: (df["protocol"]=='PCR_mass_testing') &\
                     (df["delay"]==1),'scenario'] = "PCR - 1 day delay"
dfall.loc[lambda df: (df["protocol"]=='PCR_mass_testing') &\
                     (df["delay"]==2),'scenario'] = "PCR - 2 day delay"

print(sum(dfall['scenario'] == "LFD"))
print(sum(dfall['scenario'] == "PCR - 0 delay"))
print(sum(dfall['scenario'] == "PCR - 1 day delay"))
print(sum(dfall['scenario'] == "PCR - 2 day delay"))


In [ ]:
sns.set_color_codes()

CL = 'C0'
CP0 = "C1"
CP1 = "C1"
CP2 = "C1"

paltest={'LFD':CL, 'PCR - 0 delay':CP0, 'PCR - 1 day delay':CP1, 'PCR - 2 day delay':CP2}

sp = sns.catplot(x="tperiod", y="outbreak",hue='scenario',col='new_comply_prob',\
                 data=dfall,kind='point',palette=paltest, legend=False, \
                 hue_order=['PCR - 0 delay', 'PCR - 1 day delay', 'PCR - 2 day delay','LFD'],\
                 markers=["x", "o","+","s"], linestyles=["--", "-",":","-"])
sp.set(xlabel='Days between tests',ylim=(0,0.25))
ax = sp.axes.flatten()
ax[0].set_ylabel('Outbreak probability')
lab = ['a','b','c']
for i in range(3):
    ax[i].set_title('(%s) Extra compliance prob. = %.1f'%(lab[i],NCP[i]))

ax[0].legend(loc='lower right',title='Scenario',borderpad=0.1,labelspacing=0.25)
plt.tight_layout()
mean_baseline = np.mean(dfball['outbreak'])
for i in np.arange(3):
    ax[i].plot(Tperiod.astype("|S3"),mean_baseline*np.ones(len(Tperiod)),c='k')
sp.savefig('test_outbreak_parcel.png')

In [ ]:
sp = sns.catplot(x="tperiod", y="NInfected",hue='scenario',col='new_comply_prob',data=dfall,kind='point',
                palette=paltest, legend=False, \
                 hue_order=['PCR - 0 delay', 'PCR - 1 day delay', 'PCR - 2 day delay','LFD'],\
                 markers=["x", "o","+","s"], linestyles=["--", "-",":","-"])
sp.set(xlabel='Days between tests', ylabel='Secondary cases',ylim=(0,3.5))
ax = sp.axes.flatten()
ax[0].legend(loc='lower right',title='Scenario',borderpad=0.1,labelspacing=0.25)
mean_baseline = np.mean(dfball['NInfected'])
for i in np.arange(3):
    ax[i].plot(Tperiod.astype("|S3"),mean_baseline*np.ones(len(Tperiod)),c='k')

In [ ]:
dfNCP1 = dfall.loc[lambda DF: (DF['scenario'] == 'LFD') & \
           ((DF['tperiod'] == 2) | (DF['tperiod'] == 4) | \
            (DF['tperiod'] == 8) | (DF['tperiod'] == 14)) &\
           (DF['new_comply_prob']==1)]
dfball['tperiod'] = 'No Testing'
dfNCP1 = pd.concat([dfNCP1, dfball])
sp = sns.catplot(x="tperiod", y="NInfected", data=dfNCP1, kind='boxen',height=4, aspect=8/4)
sp.set(xlabel='Days between tests',ylabel='Secondary cases',ylim=(0,50))

In [ ]:
Nbins = np.arange(-0.5,50.5,1)
fig,ax = plt.subplots(figsize=(9,3))

ax.hist(dfball['NInfected'],bins=Nbins,label='No testing',density=True)
ax.hist(dfNCP1['NInfected'][dfNCP1['tperiod'] == 8],bins=Nbins,alpha=0.6,label='8 day LFD',density=True)
ax.hist(dfNCP1['NInfected'][dfNCP1['tperiod'] == 4],bins=Nbins,alpha=0.6,label='4 day LFD',density=True)
ax.hist(dfNCP1['NInfected'][dfNCP1['tperiod'] == 2],bins=Nbins,alpha=0.6,label='2 day LFD',density=True)
ax.legend()
ax.set_xlabel('Secondary cases')
ax.set_ylabel('Fraction of simulations')
ax.set_ylim(1E-4,1)
ax.set_yscale('log')

In [ ]:
Nbins = np.arange(-0.5,30.5,1)
plt.hist(dfball['NInfected'],bins=Nbins,weights=(1/7.0)*np.ones(len(dfball)))
plt.yscale('log')